In [1]:
'''
SCRIPT PER PROVARE configurazioni SVM
'''

import time
t_orig = time.process_time()
last_time = 0

import warnings
warnings.simplefilter("ignore")

import codecs
import csv
import argparse
import os
import numpy as np
import pickle
import pandas as pd

from time import time
from dill import dill
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, classification_report
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import cross_validation, svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# modulo per confrontare il risultato delle regex
# richiede entrambi i db già etichettati
import tortellino_parmigiano 
import crea_db


import evaluation_report # stampa confusion_matrix e altro
from text_tokenizer import get_tokenizer 
from nltk.stem.snowball import ItalianStemmer

/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ita_stopwords = set(stopwords.words('italian'))
ita_stemmer = ItalianStemmer()
ita_stemming = ita_stemmer.stem
def get_tokenizer1(doc): #(word_ngrams=None, char_ngrams=None, stopwords=None, additional_extractors=None):
    # rimuove le stopwords
    doc = filter(lambda x: x not in ita_stopwords, word_tokenize(doc))
    # stemma le parole
    doc = [ita_stemmer.stem(a) for a in doc]
    return doc

def get_tokenizer(doc): #(word_ngrams=None, char_ngrams=None, stopwords=None, additional_extractors=None):
    doc = filter(lambda x: x not in ita_stopwords, word_tokenize(doc))
    #doc = map(ita_stemming, doc)
    #doc = [ita_stemmer.stem(a) for a in doc]
    temp = ''
    for par in doc:
        temp = '{} {}'.format(temp, ita_stemming(par))
    return temp

In [3]:
DATA_FOLDER = "/home/backup/dati_e_csv/" # DATI
# ci assicuriamo siano presenti i DB
training_db_name = crea_db.build_training(OUTPUT=DATA_FOLDER)
test_db_name = crea_db.build_test(OUTPUT=DATA_FOLDER) ################################
PATH = '/home/backup/dati_e_csv/training_stemmed.pickle'
    
if os.path.exists(PATH):
    print('Trovato salvataggio training set')
#     with open('/home/backup/dati_e_csv/training_stemmed.pickle', 'rb') as file:
#         salvataggio = pickle.load(file)
#     texts = salvataggio.features
#     texts_stemmed = salvataggio.stemmed
#     Y = salvataggio.labels
    salvataggio = pd.read_pickle(PATH)
    etichette = salvataggio[
        salvataggio.columns.difference(['testo', 'testo_stemmato'])].columns.values.tolist()
    print('Salvataggio caricato')
else:
    print('Creo il training set')
    
    texts = list()
    labels = list()
    names= []

    # load training dataset # possiamo usare sqlite
    with codecs.open(os.path.join(DATA_FOLDER, training_db_name), 'r', errors='ignore') as file:
        reader = csv.reader(file, delimiter=';')
        etichette = next(reader)
        for row in reader:
            # load documents
            #print('Nome {}, etichette {}, conversazione{}'.format(row[10], row[:10], row[11]+row[12]))
            #raise SystemExit
            unito = (row[11]+row[12]).replace('#O:', '').replace('#C:', '') #concatena il testo cliente con operatore
            texts.append(unito)
            # load row name
            names.append(row[10])
            # load labels
            labels.append(list(map(lambda x: int(x), row[:10])))

    print('dimensioni:',len(texts), len(labels))
    # tokenization function # possiamo usare anche solo un canale cliente/operatore
    print('Inizio lo stemming')
    etichette = etichette[:10]
    salvataggio = pd.DataFrame()
    salvataggio['testo'] = texts
    salvataggio[etichette] = pd.DataFrame(Y)
    salvataggio['testo_stemmato'] = pd.Series(list(map(get_tokenizer, texts)))
    print('Training set creato e salvato')
    
    # save DataFrame
    salvataggio.to_pickle(PATH)
    
#     salvataggio = CestinoTraining(texts, Y, etichette)
#     texts_stemmed = salvataggio.stemmed
#     Y = salvataggio.labels
#     with open('/home/backup/dati_e_csv/training_stemmed.pickle', 'wb') as file:
#         pickle.dump(salvataggio, file, protocol=0)

Training sqlite e csv presenti!
Test sqlite e csv presenti!
Trovato salvataggio training set
Salvataggio caricato


In [5]:
salvataggio[salvataggio.columns.difference(['testo', 'testo_stemmato'])].columns.values.tolist()

['attivazione',
 'canone rai',
 'cessazione',
 'contratto',
 'credito',
 'domiciliazione',
 'fatturazione',
 'gr-invito a chiamare - focus',
 'non di competenza',
 'richiamate']

In [4]:
salvataggio.head()

,testo,attivazione,canone rai,cessazione,contratto,credito,domiciliazione,fatturazione,gr-invito a chiamare - focus,non di competenza,richiamate,testo_stemmato
0,sono geloso macchina perché che questo mi ...,0,0,0,0,1,0,0,1,0,0,gelos macchin dann numer nient quand prossim ...
1,sì hanno hanno inviato documentazioni fonte...,0,0,0,0,0,0,0,1,0,0,sì invi document font signor numer client ved...
2,io la sento malissimo sì ma la bolletta il...,0,0,0,0,0,0,0,0,0,0,sent malissim sì bollett numer civic dic codi...
3,spetti si senta lei si esprime ma il moti...,0,0,0,0,1,0,0,1,0,1,spett sent esprim mot collett dic volt chiam ...
4,sì per attivare che il contratto segnalasse...,0,0,0,0,0,0,0,1,0,0,sì attiv contratt segnal ' abit richiest può ...


In [7]:
import time
print('Time to load create the model: ', time.process_time()-last_time)
last_time = time.process_time()

Time to load create the model:  5.437809368


In [5]:
# some features selection
# stopwords, tf-idf
X_traing = salvataggio['testo_stemmato']
count_vect = CountVectorizer(analyzer='word', stop_words=ita_stopwords, 
                             min_df=5, ngram_range=(1,2))
print('count_vec')
X_vec = count_vect.fit_transform(X_traing)
print('tfidf')
tf_transformer = TfidfTransformer(use_idf=True)
X_train_tf = tf_transformer.fit_transform(X_vec)
print('done')

In [6]:
# statistical feature selection with chi2
# rank features according to the average or the maximum Chi-squared score across all labels
# led to most of the best classifiers while using less features. 
# fonte(http://ceur-ws.org/Vol-1094/bioasq2013_submission_8.pdf)
Y_train = salvataggio[etichette]
# the matrices of selected features, by category
selected_features = [] 
chi2_model = []
for label in etichette:
    chi2_model.append(SelectKBest(chi2, k=1500))
    selected_features.append(chi2_model[-1].fit_transform(X_train_tf, Y_train[label]))
    
    #selected_features.append(list(selector.scores_)) # featuresx10 matrix, score of each featrue for each class
print('selected_features:',len(selected_features), selected_features[1].shape)
print('total features:', X_train_tf.shape)

# prendo dalla matrice X solo le feature che hanno un max > di una threshold

selected_features: 10 (24379, 1500)
total features: (24379, 174028)


In [7]:
# SVM
algo = LinearSVC(C=20)#, class_weight={0:0.2,1:1})
models = [algo] * len(etichette)

for i in range(len(etichette)):
    models[i].fit(selected_features[i], Y_train[etichette[i]])
    print('ok', i)

ok 0
ok 1
ok 2
ok 3
ok 4
ok 5
ok 6
ok 7
ok 8
ok 9


In [8]:
# importo il test set per la valutazione del modello
unito_test = []
#label_names = []
test_texts = list()
test_labels = list()
#raise SystemExit

# load test data
# carica il test set
with codecs.open(os.path.join(DATA_FOLDER, test_db_name), 'r', errors='ignore') as file:
    reader = csv.reader(file, delimiter=';')
    etichette = next(reader)[3:]
    print(etichette)
    for row in reader:
        # load documents
        test_texts.append((row[1]+row[2]).replace('O:', '').replace('C:', ''))
        # load names
        #names_test.append(row[0])
        # load labels
        test_labels.append(list(map(lambda x: int(x),row[-10:])))
        
X_test = list(map(get_tokenizer, test_texts))
print('ok')

['attivazione', 'canone rai', 'cessazione', 'contratto', 'credito', 'domiciliazione', 'fatturazione', 'gr-invito a chiamare - focus', 'non di competenza', 'richiamate']
ok


In [9]:
# apply model to data
Y_test = pd.DataFrame(test_labels, columns=etichette)
Y_test.head()

test_doc_term = count_vect.transform(X_test)
test_doc_idf = tf_transformer.transform(test_doc_term)
prediction = np.zeros(Y_test.shape)
for i in range(len(etichette)):
    chi2_test = chi2_model[i].transform(test_doc_idf)
    prediction[:,i] = models[i].predict(chi2_test) #np_array

print(Y_test.shape)
# print evaluation report
print('Report:')
print(classification_report(Y_test, prediction, target_names=etichette))
#evaluation_report.print_evaluation(Y_test, predicted_labels)
# conta quante etichette sono state messe
#print(colonna, np.sum(predicted_labels))
#print('Decision function: ')

#scores = algo.decision_function(test_texts) #np_array

print('\n')
print('accuracy', accuracy_score(Y_test, prediction))
print('precision micro',precision_score(Y_test, prediction, average='micro') )
print('recall micro',recall_score(Y_test, prediction, average='micro'))
print('f1 micro', f1_score(Y_test, prediction, average='micro'))
print('precision macro', precision_score(Y_test, prediction, average='macro') )
print('recall macro', recall_score(Y_test, prediction, average='macro'))
print('f1 macro', f1_score(Y_test, prediction, average='macro'))

(424, 10)
Report:
                              precision    recall  f1-score   support

                 attivazione       0.18      1.00      0.30        73
                  canone rai       0.31      0.92      0.46        63
                  cessazione       0.06      0.60      0.11        10
                   contratto       0.18      1.00      0.31        74
                     credito       0.39      1.00      0.56       162
              domiciliazione       0.07      1.00      0.13        21
                fatturazione       0.30      0.99      0.46       124
gr-invito a chiamare - focus       0.05      1.00      0.10        23
           non di competenza       0.15      0.72      0.25        50
                  richiamate       0.00      0.00      0.00         3

                 avg / total       0.26      0.96      0.40       603



accuracy 0.00235849056604
precision micro 0.196855775803
recall micro 0.955223880597
f1 micro 0.326438084443
precision macro 0.1686246662

In [ ]:
# confronto gli error rate per ciascuna classe
# predico il training set
Y_pred_train = model.predict(texts)

accuracy_test = []
accuracy_train = []
for i in range(predicted_labels.shape[1]):
    accuracy_test.append(accuracy_score(Y_test[:,i], predicted_labels[:,i]))
    accuracy_train.append(accuracy_score(Y[:,i], Y_pred_train[:,i]))
    

print ('error rate:')
print('{:>35} {:>10}'.format('test','train'))
for n in range(len(etichette[:10])):
      print('{:>23}{:>12}{:>10}'.format(etichette[n], round(1-accuracy_test[n],3), round(1-accuracy_train[n],3)))
    
print('''
    COME INTERPRETARE:
    
    Training error is low but is much lower than testing error - overfitting
    Both errors are low - ok
    Both errors are high - underfitting
    Training error is high but testing is low - error in implementation or very small dataset
''')